In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import spotipy
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score as r2
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.models import Sequential
from configparser import ConfigParser
from spotipy.oauth2 import SpotifyClientCredentials
from numpy import matrix
from get_timbre import get_timbre
from evaluation import evaluate_data
from termcolor import colored as cl

In [13]:
# Setup dataset

data = pd.read_csv("./data/data_timbre.csv")

labels = data['order']
columns_to_remove = ['Unnamed: 0', 'Unnamed: 0.1', 'key', 'mode', 'id', 'duration_ms', 'time_signature', 'track_title', 'album_title', 'album_artist', 'track_number', 'total_tracks', 'order']
data.drop(columns = columns_to_remove, axis = 1, inplace = True)

data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']] = MinMaxScaler().fit_transform(data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']])

training_set, test_set, training_labels, test_labels = train_test_split(data, labels, test_size=0.01, random_state=42)

# data
training_set

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_timbre,song_timbre_start,song_timbre_end,loudness_start,loudness_end
1162,0.441,0.7440,0.818804,0.4760,0.262000,0.000382,0.1380,0.1240,0.328130,0.481316,0.432125,0.358045,0.639406,0.801207
287,0.612,0.9890,0.880760,0.3670,0.006890,0.114000,0.2010,0.1050,0.547432,0.374123,0.742210,0.368176,0.455139,0.074779
856,0.726,0.7600,0.906988,0.3470,0.282000,0.000210,0.1580,0.5720,0.483415,0.307475,0.735916,0.932930,0.363980,0.649394
373,0.077,0.0977,0.306369,0.0374,0.716000,0.885000,0.0705,0.0372,0.324583,0.360078,0.373066,0.865443,0.304430,0.152293
2646,0.482,0.4150,0.815459,0.0291,0.011200,0.003200,0.0846,0.2050,0.540643,0.346650,0.616537,0.479745,0.823653,0.094808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.383,0.6210,0.830482,0.1130,0.027200,0.000000,0.4270,0.2130,0.322708,0.261726,0.653905,1.000000,0.406734,0.688868
466,0.833,0.8770,0.907398,0.2160,0.005890,0.121000,0.6690,0.0380,0.515920,0.211869,0.532917,0.799374,0.560645,0.618584
3092,0.491,0.9000,0.867725,0.0362,0.000323,0.036400,0.3200,0.2860,0.499903,0.210321,0.462922,0.522600,0.823651,0.302241
3772,0.540,0.4180,0.667308,0.0306,0.664000,0.000387,0.1060,0.2840,0.543252,0.243174,0.226151,0.297094,0.681551,0.934780


In [14]:
# Setup autoencoder

encoder_layers = [
            layers.Dense(3, activation='relu', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9)),
            ]

decoder_layers = [
            layers.Dense(14, activation='sigmoid', activity_regularizer=regularizers.l1_l2(l1=1e-10, l2=1e-9))
            ]

encoder = Sequential(encoder_layers)
decoder = Sequential(decoder_layers)

autoencoder = Sequential([encoder, decoder])

# autoencoder.compile(optimizer='adam', loss=losses.BinaryCrossentropy())
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [15]:
autoencoder.fit(training_set, training_set,
                epochs=100,
                shuffle=True,
                validation_split = 0.2)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

123/123 [==============================] - 1s 6ms/step - loss: 0.0826 - val_loss: 0.0753
Epoch 2/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0664 - val_loss: 0.0562
Epoch 3/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0496 - val_loss: 0.0451
Epoch 4/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0444 - val_loss: 0.0427
Epoch 5/100
123/123 [==============================] - 0s 2ms/step - loss: 0.0426 - val_loss: 0.0411
Epo

In [16]:
encoded_tracks = encoder.predict(test_set)
decoded_tracks = decoder.predict(encoded_tracks)

distances = []
for i in range(len(decoded_tracks)):
    distances.append(np.linalg.norm(decoded_tracks[i]-test_set.iloc[i]))
np.mean(distances)

# autoencoder.evaluate(test_set)
# autoencoder.summary()

0.4642946823575819

In [17]:
autoencoder.save('./regression/autoencoder.h5')
encoder.save('./regression/encoder.h5')
decoder.save('./regression/decoder.h5')